In [1]:
import cv2
import numpy as np
import random as rng
from ipynb.fs.full.wavelet import Wavelet

c:\Users\lc100\AppData\Local\miniconda3\envs\frameSubtraction\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\lc100\AppData\Local\miniconda3\envs\frameSubtraction\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\lc100\AppData\Local\miniconda3\envs\frameSubtraction\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def display(frame,name=None):
    """ resize and display a frame

    Args:
        frame (array): frame that will be displayed
        name (string, optional): name of the window. Defaults to None.

    Returns:
        arrray: frame resized
    """
    scale_percent = 60 # percent of original size
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    dim = (width, height)

    # resize image
    resized = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    
    if name is not None:
        cv2.imshow(winname=name, mat=resized)
        
    return resized

######################################################################################################

def drawContour(mask,kernelsize=15):
    """take a mask and draw the contour around the objects

    Args:
        mask (array): Mask with objects on it
        kernelsize (int, optional): size of the kernel. Defaults to 15.

    Returns:
        _type_: _description_
    """
    
    kernel6 = np.ones((kernelsize,kernelsize), np.uint8)
    dil = cv2.dilate(mask, kernel6, iterations=1)
    contours, _ = cv2.findContours(dil, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    return dil,contours


######################################################################################################

def convert(img, target_type_min, target_type_max, target_type):
    imin = img.min()
    imax = img.max()

    a = (target_type_max - target_type_min) / (imax - imin)
    b = target_type_max - a * imax
    new_img = (a * img + b).astype(target_type)
    
    return new_img

######################################################################################################

def correlation_coefficient(patch1, patch2):
    product = np.mean((patch1 - patch1.mean()) * (patch2 - patch2.mean()))
    stds = patch1.std() * patch2.std()
    if stds == 0:
        return 0
    else:
        product /= stds
        return product
    
######################################################################################################
    
def boundingBoxes(frame_diff,contours,show=False):
    """ put object in the smallest box possible

    Args:
        frame_diff (array): mask of the objects
        contours (list): contour of the object
        show (bool, optional): to see all the boxes. Defaults to False.

    Returns:
        _type_: _description_
    """
    
    contours_poly = [None]*len(contours)
    boundRect = [None]*len(contours)
    
    for i, c in enumerate(contours):
        contours_poly[i] = cv2.approxPolyDP(c, 5, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])
    
    drawing = np.zeros((frame_diff.shape[0], frame_diff.shape[1], 3), dtype=np.uint8)
    
    for i in range(len(contours)):
        color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
        cv2.drawContours(drawing, contours_poly, i, color)
        cv2.rectangle(drawing, (int(boundRect[i][0]), int(boundRect[i][1])), \
          (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)
    
    if show:
        display(drawing,name='all boxes')

    return frame_diff, boundRect

######################################################################################################

def checkBoxes(frame_diff, boundingBoxes ,show=False,coeff=150,wave_temp=None,wave_temp2=None):
    """ select the ball among all the objects by using the wavelet transform

    Args:
        frame_diff (array): _description_
        boundingBoxes (list): boxes of all the objects
        show (bool, optional): to see the boxe selected if any. Defaults to False.
        coeff (int, optional): coeff of similitude maximum allowed. Defaults to 150.
        wave_temp (array, optional): wavelet transform of the template. Defaults to None.
        wave_temp2 (array, optional): wavelet transform of the black template. Defaults to None.

    Returns:
        array: best box
    """
    
    if wave_temp is None:
        print("error no template checkBoxes ")
        return None
    if wave_temp2 is None:
        wave_temp2=wave_temp
    
    # we don't consider boxes that are too small or too big
    size_min=32
    size_max=wave_temp.shape[0]*4
    
    maxBox=None 
    #totH=0    

    for box in boundingBoxes:

        if box[3]>size_min and box[3]<size_max and box[2]>size_min and box[2]<size_max and box[2]/box[3]<100 and box[2]/box[3]>0.001:

            box_cp = frame_diff[box[1]:(box[1]+box[3]),box[0]:(box[0]+box[2]),:]
                       
            #temp_hist0 = cv2.calcHist(box_cp, [0], None,[256],[0,256] )
            #temp_hist1 = cv2.calcHist(box_cp, [1], None,[256],[0,256] )
            #temp_hist2 = cv2.calcHist(box_cp, [2], None,[256],[0,256] )
            #tot_histogramm=np.sum(temp_hist0[120:130]+temp_hist1[120:130]+temp_hist2[120:130])
            
            wave_box=Wavelet(box_cp)

            diff1=cv2.absdiff(wave_temp[:2,:2,:],wave_box[:2,:2,:])#cv2.absdiff(wave_temp[0,0,:],wave_box[0,0,:])+cv2.absdiff(wave_temp[1,1,:],wave_box[1,1,:])
            diff2=cv2.absdiff(wave_temp2[:2,:2,:],wave_box[:2,:2,:])#cv2.absdiff(wave_temp2[0,0,:],wave_box[0,0,:])+cv2.absdiff(wave_temp2[1,1,:],wave_box[1,1,:])
            diff=min(np.sum(diff1),np.sum(diff2))
        
            diff=np.sum(diff)
            
            diff_weight=( diff/(3) )#+tot_histogramm
            
            if show:
                cv2.imshow(winname="selected boxe", mat=box_cp)
                print("diff: ",diff/3)
                
            
            if diff_weight is not None and coeff>diff_weight:
                coeff=diff_weight
                maxBox=box
                #totH=tot_histogramm

    if maxBox is None:
        return None,None
    
    print(coeff)
    
    #if show:
    #    box_cp= frame_diff[maxBox[1]:(maxBox[1]+maxBox[3]),maxBox[0]:(maxBox[0]+maxBox[2]),:]   
    #    cv2.rectangle(frame_diff, (int(box[0]), int(box[1])),(int(box[0]+box[2]), int(box[1]+box[3])), (255,255,255), 2)        
    #    display(frame_diff,name='select box')
        
    return frame_diff,maxBox

######################################################################################################

def substraction(y0,y1,blur_type=0,threshold_type=0,threshold=50,blur=29,show=False,erode=0,dilate=0):
    """_summary_

    Args:
        y0 (array): one dimension of the frame 0
        y1 (array): one dimension of the frame 0
        blur_type (int, optional): 0= gaussian 1= median. Defaults to 0.
        threshold_type (int, optional): 0=binary 1=tozero 2=adaptive threshold. Defaults to 0.
        threshold (int, optional):  Defaults to 50.
        blur (int, optional): parameter for the blur. Defaults to 29.
        show (bool, optional):  Defaults to False.

    Returns:
        _type_: _description_
    """

    if blur_type==0:
        gb0=cv2.GaussianBlur(y0,(blur,blur),0)
        gb1=cv2.GaussianBlur(y1,(blur,blur),0)
    else:
        gb0=cv2.medianBlur(y0, blur)
        gb1=cv2.medianBlur(y1, blur)
    
    diff0=cv2.absdiff(gb0,gb1)#+cv2.absdiff(ub0,ub1)+cv2.absdiff(vb0,vb1)
    diff1=cv2.absdiff(gb1,gb0)#+cv2.absdiff(ub1,ub0)+cv2.absdiff(vb1,vb0)
        
    if threshold_type==0:
        _, diff0 = cv2.threshold(diff0, threshold, 255, cv2.THRESH_BINARY)
        _, diff1 = cv2.threshold(diff1, threshold, 255, cv2.THRESH_BINARY)
    elif threshold_type==1:
        _, diff0 = cv2.threshold(diff0, threshold, 255, cv2.THRESH_TOZERO  )
        _, diff1 = cv2.threshold(diff1, threshold, 255, cv2.THRESH_TOZERO  )
    elif threshold_type==2:
        diff0 = cv2.adaptiveThreshold(diff0, 255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 31, 5)
        diff1 = cv2.adaptiveThreshold(diff1, 255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 31, 5)
    
    frame_diff=cv2.add(diff0,diff1)
    frame_diff = cv2.erode(frame_diff, None, iterations=erode)
    frame_diff = cv2.dilate(frame_diff, None, iterations=dilate)
     
    if show:
        display(frame_diff,name='substraction1')

    return frame_diff
    
